In [0]:
%run ../../_utils

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import when, to_date, col, dayofweek
from pyspark.sql.types import StringType, BooleanType


# Camada Silver

Na camada silver, limpezas e ajustes em dados devem ser aplicados

Caso seja possível, enriquecer os dados e extrair dados também deve acontecer nessa camada (minha definição)

In [0]:
tb_name = "silver.olist_sellers"
dataset_location = "olist_sellers_dataset"
target_location = f"dbfs:/FileStore/delta/silver/brazilian_ecommerce/{dataset_location}"

## 1 - Data ingestion

In [0]:
df = spark.read.table("bronze.olist_sellers") # leituira da delta table

In [0]:
df.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



In [0]:
display(df.take(10))

seller_id,seller_zip_code_prefix,seller_city,seller_state
3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
c240c4061717ac1806ae6ee72be3533b,20920,rio de janeiro,RJ
e49c26c3edfa46d227d5121a6b6e4d37,55325,brejao,PE
1b938a7ec6ac5061a66a3766e0e75f90,16304,penapolis,SP
768a86e36ad6aae3d03ee3c6433d61df,1529,sao paulo,SP
ccc4bbb5f32a6ab2b7066a4130f114e3,80310,curitiba,PR



## 2 - Data Cleaning

Aqui nessa table os dados já estão (quase) limpos e tratados. Somente iremos realizar um ajuste no outlier seller_city "04482255"

In [0]:
df = df.filter("seller_city != '04482255'")


## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)

[LOG] Saving silver.olist_sellers delta on dbfs:/FileStore/delta/silver/brazilian_ecommerce/olist_sellers_dataset... OK!



## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(table_name=tb_name, target_location=target_location)

[LOG] Creating delta table silver.olist_sellers on dbfs:/FileStore/delta/silver/brazilian_ecommerce/olist_sellers_dataset... OK!


In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql
select
  seller_city,
  count(*) as total_hits
from
  silver.olist_sellers
group by
  seller_city
limit
  10